Since by now you have all written some Python code that works, we are going to jump directly into doing some science tasks with data pulled directly from real sources, instead of using a dataset we cleaned for you.

Some of the stuff we are going to do will be fairly advanced but I think that seeing how the language can automate your workflow - and especially how it can handle the tasks that are annoying or difficult - will make it more likely that you'll come back to it. We are throwing a lot at you, so put a red sticky up if you need hands-on help and interrupt if you have a question, and we'll adjust as we go.

We are interested in the comparing the discharge over a specific time period at different points along the Colorado River. The traditional way to do this is through the USGS website:

http://waterwatch.usgs.gov/?m=real&r=az

and then precisely selecting the station you want and going through the whole process by hand. At the end, you have end up with some csv files that look like this:

http://waterdata.usgs.gov/az/nwis/uv?cb_00060=on&cb_00065=on&format=rdb&site_no=09380000&period=&begin_date=2016-01-01&end_date=2016-01-10

The header is annoying because it doesn't always have the same number of lines. Depending on the specific options we selected, the table has a different number of columns. Automating the analysis of these files is rough, and it requires quite a bit of handholding in Excel. If we were to do this frequently or for many stations, the chances of introducing an error are high.

Instead of doing this by hand, we are going to request the data through the USGS web services. Web services are automated tools for transfering data directly from machine to machine. Effectively, the code we write will ask a server for some data, and that server will hand it back to your code. The way that these commands and data are transferred is through an API (Application Programming Interface), which is essertially a set of functions and protocols for that interaction. We don't actually need to understand in detail what any of that means to use it, though - help pages are our friend.

Let's start by getting data from the streamgage station at Lee's Ferry. We'll make a variable with the station number (which I looked up):

In [1]:
station = 09380000

SyntaxError: invalid token (<ipython-input-1-6f8681f3257f>, line 1)

An integer cannot start with a zero, so Python doesn't know what we mean and produces a Syntax Error. Let's rewrite the station number as a string by putting it in quotes (single or double both work):

In [2]:
station = '09380000'

We are interested in data between two specific dates:

In [6]:
start_date = '2016-01-01'
end_date = '2016-01-10'

Data is accesed through APIs using URLs that contain the different pieces of information that the server needs to identify the data that we need. The USGS has a nifty little tool for helping us compose the URL:

http://waterservices.usgs.gov/rest/IV-Test-Tool.html

Let's use that tool to create the URL for the data we want:

http://waterservices.usgs.gov/nwis/iv/?format=rdb&sites=09380000&startDT=2016-01-01&endDT=2016-01-10&parameterCd=00060,00065

If we go to that URL, we could just download that file and process it manually. We are not going to do that.

Early today we used a Python library called numpy to handle arrays of data. `numpy arrays` are matrices and are great for doing calculations on data, but they can only contain floats or integers. Tabular data are best handled by spreadsheets where entries such as dates and times are in some useful format.

One of the best options for working with tabular data in Python is the Python Data Analysis Library (a.k.a. Pandas). The Pandas library provides data structures, produces high quality plots with matplotlib (the plotting library we used earlier) and integrates nicely with other libraries that can use numpy arrays.

In [18]:
url = 'http://waterservices.usgs.gov/nwis/iv/?format=rdb&sites=09380000&startDT=2016-01-01&endDT=2016-01-10&parameterCd=00060,00065'

import pandas as pd

We assigned the URL we want to the variable `url`, and imported the Pandas library and gave it the shortcut `pd`. Pandas has multiple functions for importing data into Python, and we can use both local paths (within our computers) or URLs to point to the data we want to import. Let's start by loading a file with streamgage data that has already been cleaned up:

In [19]:
pd.read_csv(url)

CParserError: Error tokenizing data. C error: Expected 1 fields in line 12, saw 2


Pandas is unhappy and cannot load our file. Pandas expects one line of column titles and then data with the same number of columns. The header in the file we want to read is making it appear like the number of columns change partway through the file.

The Pandas documentation will show us how to tell the function to ignore the header:

In [21]:
help(pd.read_csv)

Help on function read_csv in module pandas.io.parsers:

read_csv(filepath_or_buffer, sep=',', dialect=None, compression='infer', doublequote=True, escapechar=None, quotechar='"', quoting=0, skipinitialspace=False, lineterminator=None, header='infer', index_col=None, names=None, prefix=None, skiprows=None, skipfooter=None, skip_footer=0, na_values=None, true_values=None, false_values=None, delimiter=None, converters=None, dtype=None, usecols=None, engine=None, delim_whitespace=False, as_recarray=False, na_filter=True, compact_ints=False, use_unsigned=False, low_memory=True, buffer_lines=None, warn_bad_lines=True, error_bad_lines=True, keep_default_na=True, thousands=None, comment=None, decimal='.', parse_dates=False, keep_date_col=False, dayfirst=False, date_parser=None, memory_map=False, float_precision=None, nrows=None, iterator=False, chunksize=None, verbose=False, encoding=None, squeeze=False, mangle_dupe_cols=True, tupleize_cols=False, infer_datetime_format=False, skip_blank_lines=

A way to see the documentation as a separate window within an iPython notebook is with a question mark:

In [22]:
pd.read_csv?

The function `read_csv` requires the first parameter (a filepath) and accepts many optional parameters (if not set, they use a default value). For example, `sep=','` is an optional function parameter that defines the column separator. By default, it's a comma.

Let's try loading our file again. Because the columns in our file are separated by tabs, we need to set the `sep` parameter. To remove the header, we will also tell Pandas that lines that start with `#` are comments and it should not import them.

In [23]:
pd.read_csv(url, sep='\t', comment='#')

,agency_cd,site_no,datetime,tz_cd,01_00060,01_00060_cd,02_00065,02_00065_cd
0,5s,15s,20d,6s,14n,10s,14n,10s
1,USGS,09380000,2016-01-01 00:00,MST,15200,P,9.74,P
2,USGS,09380000,2016-01-01 00:15,MST,14900,P,9.67,P
3,USGS,09380000,2016-01-01 00:30,MST,14600,P,9.62,P
4,USGS,09380000,2016-01-01 00:45,MST,14200,P,9.55,P
5,USGS,09380000,2016-01-01 01:00,MST,14000,P,9.49,P
6,USGS,09380000,2016-01-01 01:15,MST,13600,P,9.42,P
7,USGS,09380000,2016-01-01 01:30,MST,13400,P,9.37,P
8,USGS,09380000,2016-01-01 01:45,MST,13000,P,9.28,P
9,USGS,09380000,2016-01-01 02:00,MST,12700,P,9.23,P


There's our data! The first row (remember that Python starts indexing at 0) doesn't belong to the data, so we can use the parameter `skiprows` to ignore it:

In [48]:
data = pd.read_csv(url, header=1, sep='\t', comment='#')

In [49]:
data.head()

,5s,15s,20d,6s,14n,10s,14n.1,10s.1
0,USGS,9380000,2016-01-01 00:00,MST,15200,P,9.74,P
1,USGS,9380000,2016-01-01 00:15,MST,14900,P,9.67,P
2,USGS,9380000,2016-01-01 00:30,MST,14600,P,9.62,P
3,USGS,9380000,2016-01-01 00:45,MST,14200,P,9.55,P
4,USGS,9380000,2016-01-01 01:00,MST,14000,P,9.49,P
